## Тема “Создание признакового пространства”

### Продолжим обработку данных с Твиттера. 

In [1]:
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec

import warnings 
warnings.filterwarnings("ignore")
import os

1. Создайте мешок слов с помощью sklearn.feature_extraction.text.CountVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.

 -	Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
 -	Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
 -	Исключим стоп-слова с помощью stop_words='english'.
 -	Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью CountVectorizer.get_feature_names().

In [2]:
processed_df = pd.read_pickle('processed_df.pkl')
processed_df.head(2)

,id,label,tweet,clean_tweet,tweet_token,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee..."


In [3]:
vectorizer = CountVectorizer(max_df=0.9, stop_words='english', max_features=1000)
stemmed = processed_df['tweet_stemmed'].apply(lambda x: ' '.join(x))
BOW_stemmed = vectorizer.fit_transform(stemmed)
feature_names = vectorizer.get_feature_names()
pd.DataFrame(BOW_stemmed.toarray(), columns = feature_names)

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,yo,yoga,york,young,youtub,yoyou,yoyour,yoyoy,yr,yummi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49154,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49155,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49156,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49157,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
vectorizer = CountVectorizer(max_df=0.9, stop_words='english', max_features=1000)
lemmatized = processed_df['tweet_lemmatized'].apply(lambda x: ' '.join(x))
BOW_lemmatized = vectorizer.fit_transform(lemmatized)
feature_names = vectorizer.get_feature_names()
pd.DataFrame(BOW_lemmatized.toarray(), columns = feature_names)

,able,absolutely,account,act,action,actor,actually,adapt,add,adventure,...,yo,yoga,york,young,youtube,yoyou,yoyour,yoyoyou,yr,yummy
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49154,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49155,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49156,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49157,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


2. Создайте мешок слов с помощью sklearn.feature_extraction.text.TfidfVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.
 -	Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
 -	Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
 -	Исключим стоп-слова с помощью stop_words='english'.
 -	Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью TfidfVectorizer.get_feature_names().

In [5]:
vectorizer = TfidfVectorizer(max_df=0.9, stop_words='english', max_features=1000)
BOW_stemmed = vectorizer.fit_transform(stemmed)
feature_names = vectorizer.get_feature_names()
pd.DataFrame(BOW_stemmed.toarray(), columns = feature_names)

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,yo,yoga,york,young,youtub,yoyou,yoyour,yoyoy,yr,yummi
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.744148,0.392399,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49154,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
49155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
49156,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
49157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0


In [6]:
vectorizer = TfidfVectorizer(max_df=0.9, stop_words='english', max_features=1000)
BOW_lemmatized = vectorizer.fit_transform(lemmatized)
feature_names = vectorizer.get_feature_names()
pd.DataFrame(BOW_lemmatized.toarray(), columns = feature_names)

,able,absolutely,account,act,action,actor,actually,adapt,add,adventure,...,yo,yoga,york,young,youtube,yoyou,yoyour,yoyoyou,yr,yummy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.742113,0.39173,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49154,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0
49155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0
49156,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0
49157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0


3. Натренируем gensim.models.Word2Vec модель на наших данных.
 -	Тренировать будем на токенизированных твитах combine_df['tweet_token']
 -	Установим следующие параметры: size=200, window=5, min_count=2, sg = 1, hs = 0, negative = 10, workers= 32, seed = 34.
 -	Используем функцию train() с параметром total_examples равным длине combine_df['tweet_token'], количество epochs установим 20.

In [7]:
X_train = processed_df['tweet_token'].tolist()

model = Word2Vec(
    X_train, 
    size=200, 
    window=5, 
    min_count=1, 
    sg = 1, 
    hs = 0, 
    negative = 10, 
    workers= 32, 
    seed = 34)

total_examples = len(X_train)
model.train([["hello", "world"]], total_examples=total_examples, epochs=20)

(40, 40)

4. Давайте немного потестируем нашу модель Word2Vec и посмотрим, как она работает. Мы зададим слово positive = "dinner", и модель вытащит из корпуса наиболее похожие слова c помощью функции most_similar. То же самое попробуем со словом "trump".

In [8]:
model.most_similar(positive = "dinner")

[('brunch', 0.9567240476608276),
 ('bbq', 0.9558257460594177),
 ('wine', 0.9542522430419922),
 ('familytime', 0.9454008340835571),
 ('sushi', 0.9452800750732422),
 ('funday', 0.9451799392700195),
 ('drinks', 0.9421369433403015),
 ('lunch', 0.9411708116531372),
 ('cafe', 0.937766969203949),
 ('cocktails', 0.9377384185791016)]

In [9]:
model.most_similar(positive = "trump")

[('republican', 0.9025136232376099),
 ('donald', 0.8877378702163696),
 ('carl', 0.8843156695365906),
 ('suppoer', 0.8814209699630737),
 ('ally', 0.8803999423980713),
 ('paladino', 0.8777146339416504),
 ('fuhered', 0.8770227432250977),
 ('protesting', 0.8719953298568726),
 ('campaign', 0.8700079917907715),
 ('comments', 0.8692120313644409)]

5. Из приведенных выше примеров мы видим, что наша модель word2vec хорошо справляется с поиском наиболее похожих слов для данного слова. Но как она это делает? Она изучила векторы для каждого уникального слова наших данных и использует косинусное сходство, чтобы найти наиболее похожие векторы (слова).
Давайте проверим векторное представление любого слова из нашего корпуса, например "food".

In [10]:
model.wv['food']

array([-0.05344981,  0.3368922 , -0.07113878, -0.05584778, -0.15227112,
        0.2975606 ,  0.41055807, -0.2114883 ,  0.01150641, -0.23448062,
       -0.24492355, -0.0386951 ,  0.5270468 , -0.16335294,  0.00740039,
       -0.35728842,  0.10008913,  0.24275056,  0.22840348,  0.01959778,
        0.21209753,  0.20965378, -0.0591492 , -0.12082479,  0.02414964,
        0.4396961 ,  0.0124809 ,  0.71473765,  0.28967625,  0.271034  ,
       -0.06246327, -0.32737955, -0.6313767 ,  0.2453973 , -0.70584756,
        0.20805533,  0.05738099,  0.08060399, -0.05169034, -0.29433316,
        0.03417427,  0.07185368, -0.36398   , -0.03682853, -0.36880487,
       -0.08258989,  0.006033  , -0.29059532,  0.02792116,  0.52589947,
       -0.1490294 ,  0.28507245,  0.56691337, -0.05508605,  0.07529813,
       -0.07627535, -0.308133  ,  0.08630254,  0.6770666 ,  0.4912503 ,
        0.05521593, -0.03594681, -0.11677581,  0.21511173, -0.20208322,
       -0.34104538, -0.15888709, -0.41454184, -0.11777121, -0.06

6. Поскольку наши данные содержат твиты, а не только слова, нам придется придумать способ использовать векторы слов из модели word2vec для создания векторного представления всего твита. Существует простое решение этой проблемы, мы можем просто взять среднее значение всех векторов слов, присутствующих в твите. Длина результирующего вектора будет одинаковой, то есть 200. Мы повторим тот же процесс для всех твитов в наших данных и получим их векторы. Теперь у нас есть 200 функций word2vec для наших данных.

Необходимо создать вектор для каждого твита, взяв среднее значение векторов слов, присутствующих в твите. В цикле сделать:  vec += model_w2v[word].reshape((1, size))
и поделить финальный вектор на количество слов в твите.

На выходе должен получиться wordvec_df.shape = (49159, 200).

In [11]:
wordvec_df = np.zeros((total_examples, 200))
for i, tweet in enumerate(processed_df['tweet_token']):
    vec = np.zeros((1, 200))
    for word in tweet:
        vec += model.wv[word].reshape((1, 200))
    vec /= len(tweet)
    wordvec_df[i] = vec

In [12]:
wordvec_df.shape

(49159, 200)